# Stixrude-Lithgow-Bertelloni pseudo-omnicomponent phase generation
Required Python packages/modules

In [ ]:
import numpy as np
from os import path
import pandas as pd
import scipy.optimize as opt
import scipy.linalg as lin 
import scipy as sci
import sys
import sympy as sym

import matplotlib.pyplot as plt

Required ENKI modules (ignore the error message from Rubicon running under Python 3.6+)

In [ ]:
from thermoengine import coder, core, phases, model, equilibrate

### T,P, parameters and options for pseudo-phase generation

In [ ]:
T = 1300.0                  # K
P = 20000.0                 # bars
# T = 1500.0                  # K
# P = 40000.0                 # bars
test_endmember_code = False # output tests to validate solution endmember code generation
test_solution_code = False  # output tests to validate solution code generation
offset_value = 0.0          # Offset penality (in J) to destabilize pseudo-omnicomponent phase 4000
use_oxides_as_basis = False # Construct the pseudo-phase using oxides as components (False == elements)

In [ ]:
modelDB = model.Database(database='Berman')
phases = modelDB.phases
phases.pop('Liq');
# ADD extra phases (e.g. carbonates as needed here)
phases.keys()

In [ ]:
modelDB = model.Database(database='Stixrude')
phases = modelDB.phases
phases.pop('Liq');
phases.pop('H2O');
# ADD extra phases (e.g. carbonates as needed here)
phases.keys()

In [ ]:
def system_energy_landscape(T, P, phases, TOL=1e-3):
    elem_comps = []
    phs_sym = []
    endmem_ids = []
    mu = []
    for phsnm in phases:
        phs = phases[phsnm]
        
        elem_comp = phs.props['element_comp']
        abbrev = phs.abbrev
        endmem_num = phs.endmember_num
        iendmem_ids = list(np.arange(endmem_num))
        
        if phs.phase_type=='pure':
            nelem = np.sum(elem_comp)
            mu += [phs.gibbs_energy(T, P)/nelem]
            # print(nelem)
        else:
            nelem = np.sum(elem_comp,axis=1)
            # print(nelem)
            for i in iendmem_ids:
                imol = np.eye(phs.endmember_num)[i]
                mu += [phs.gibbs_energy(T, P, mol=imol,deriv={"dmol":1})[0,i]/nelem[i]]
                # print(nelem[i])
                
        endmem_ids.extend(iendmem_ids)
        phs_sym.extend(list(np.tile(abbrev,endmem_num)))
        # print(elem_comp)
        
        elem_comps.extend(elem_comp)
        # print(elem_comp)
        # print(phs)
        
    elem_comps = np.vstack(elem_comps)
    
    natoms = np.sum(elem_comps,axis=1)
    elem_comps = elem_comps/natoms[:,np.newaxis]
    
    elem_mask = ~np.all(elem_comps<TOL, axis=0)
    
    elem_comps = elem_comps[:, elem_mask]
    mu = np.array(mu)
    endmem_ids = np.array(endmem_ids)
    
    sys_elems = core.chem.PERIODIC_ORDER[elem_mask]
    return phs_sym, endmem_ids, mu, elem_comps, sys_elems

def prune_polymorphs(phs_sym, endmem_ids, mu, elem_comps, decimals=4):
    elem_round_comps = np.round(elem_comps, decimals=decimals)
        # Drop identical comps
    elem_comps_uniq = np.unique(elem_round_comps, axis=0)
    
    # uniq_num = elem_comps_uniq.shape[0]
    mu_uniq = []
    phs_sym_uniq = []
    endmem_ids_uniq = []
    for elem_comp in elem_comps_uniq:
        is_equiv_comp = np.all(elem_round_comps == elem_comp[np.newaxis,:], axis=1)
        equiv_ind = np.where(is_equiv_comp)[0]
        min_ind = equiv_ind[np.argsort(mu[equiv_ind])[0]]
        min_mu = mu[min_ind]
        assert np.all(min_mu <= mu[equiv_ind]), 'fail'
        
        mu_uniq.append(min_mu)
        phs_sym_uniq.append(phs_sym[min_ind])
        endmem_ids_uniq.append(endmem_ids[min_ind])
        
    mu_uniq = np.array(mu_uniq)
    
    return phs_sym_uniq, endmem_ids_uniq, mu_uniq, elem_comps_uniq



In [ ]:

phs_sym, endmem_ids, mu, elem_comps, sys_elems = system_energy_landscape(T, P, phases)
# display(phs_sym, endmem_ids, mu, elem_comps, sys_elems)
phs_sym_uniq, endmem_ids_uniq, mu_uniq, elem_comps_uniq = (
    prune_polymorphs(phs_sym, endmem_ids, mu, elem_comps))

min_mu_ind = np.argmin(mu_uniq)

print(sys_elems)
print(len(phs_sym))
print(phs_sym[min_mu_ind])
Nelems = len(sys_elems)
Npts = mu_uniq.size
print(Npts)

In [ ]:
def calc_eig_comps(elem_comps_uniq):
    avg_comp = np.mean(elem_comps_uniq, axis=0)
    comp_dev = elem_comps_uniq-avg_comp
    
    # u,s,vh = np.linalg.svd(comp_dev[:,:-1])
    # comp_eig = np.dot(comp_dev[:,:-1], vh.T)
    
    u,s,vh = np.linalg.svd(comp_dev)
    comp_eig = np.dot(comp_dev, vh.T)
    
    # print(s)
    return comp_eig, comp_dev, avg_comp, s, vh



In [ ]:
# elem_comps_all = np.vstack((elem_comps_uniq, comp_midpts))
# mu_all = np.hstack((mu_uniq, mu_midpts))


elem_comps_all = elem_comps_uniq
mu_all = mu_uniq


# comp_midpts=None
# mu_midpts=None
# elem_comps_all = np.vstack((elem_comps_uniq, comp_midpts))
# mu_all = np.hstack((mu_uniq, mu_midpts))


In [ ]:
elem_comps_all.shape

In [ ]:
ind_rows, ind_cols = np.tril_indices(Nelems,-1)
cross_term_inds = np.vstack((ind_rows,ind_cols))
cross_term_inds

In [ ]:
XiXj = elem_comps_all[:, cross_term_inds[0]]*elem_comps_all[:, cross_term_inds[1]]
X2_sum = np.sum(XiXj,axis=1)
X2_sum.shape

In [ ]:
# (np.sum(XiXj>0, axis=1)).shape
XiXj.shape

In [ ]:
XlogX = elem_comps_all*np.log(elem_comps_all)
XlogX[elem_comps_all==0] = 0
XlogX_sum = np.sum(XlogX,axis=1)
XlogX_sum

In [ ]:
plt.figure()
plt.plot(XlogX_sum, 'ro')
plt.plot(-3.5*X2_sum, 'kx')

x = np.linspace(-1.5,0, 10)
plt.figure()
plt.plot(XlogX_sum, -3.5*X2_sum, 'ro')
plt.plot(x,x, 'k--')

In [ ]:
# %%timeit
# comp_eig, comp_dev, avg_comp, s, vh = calc_eig_comps(elem_comps_uniq)
comp_eig, comp_dev, avg_comp, s, vh = calc_eig_comps(elem_comps_all)

In [ ]:
plt.figure()
plt.plot(s,'ko')

In [ ]:
s[-1]

In [ ]:
np.sqrt(np.sum(comp_eig[:,-1]**2))

In [ ]:
# np.mean(comp_dev,axis=0)
plt.figure()
plt.imshow(comp_eig,cmap='seismic')
# plt.figure()
# plt.imshow(np.dot(comp_dev,vh.T),cmap='seismic')

In [ ]:
plt.figure()
plt.imshow(vh,cmap='seismic')

In [ ]:
vh[-1]

In [ ]:
# avg_mu = np.mean(mu_uniq)
# X = elem_comps_uniq-avg_comp
avg_mu = np.mean(mu_all)
y = mu_all-avg_mu
Ndim = comp_eig.shape[1]

In [ ]:

# X2_term = np.sum(comp_eig**2,axis=1)

# xobs = np.hstack((comp_eig, comp_eig**2))
# xobs = np.hstack((np.ones((len(y),1)), comp_eig, comp_eig**2))

# xobs = np.hstack((comp_eig, comp_eig**2))
# xobs = np.hstack((elem_comps_uniq, XlogX_sum[:,np.newaxis]))


xobs = np.hstack((elem_comps_all, XlogX_sum[:,np.newaxis]))
# xobs = np.hstack((elem_comps_all, X2_sum[:,np.newaxis]))
# xobs = np.hstack((elem_comps_all, XiXj))

# xobs = np.hstack((comp_eig, X2_term[:,np.newaxis]))
# comp_ext = np.vstack((comp_eig, comp_midpt))
# xobs = np.hstack((comp_ext, comp_ext**2))

yexp_scl = np.floor(np.log10(np.max(y)-np.min(y)))
yscl = 10**yexp_scl
# y_ext = np.hstack((y, y_midpt))
# yobs = mu_dev_uniq/1e5

yobs = y/yscl
# yobs = y_ext/yscl
print(yscl)

In [ ]:
yscl

In [ ]:
print(xobs[0])

In [ ]:
plt.figure()
plt.plot(yobs,'o');

plt.figure()
plt.plot(xobs,'-');

In [ ]:
def reweight_fit(scl, xobs, yobs, Ndim=None, yresid=0, TOL=1e-4, param0=None):
    if np.isscalar(yresid):
        yresid = np.tile(yresid, yobs.size)
        
    err = np.ones(yobs.shape)
    mask_pos = yresid>0
    mask_neg = yresid<0
    
    yabs_dev = np.abs(yresid)
    err0 = np.median(yabs_dev)
    if err0==0:
        err0=1
    
    yabs_dev[yabs_dev<TOL] = TOL
    
    err_fac = 1/np.sqrt(yabs_dev)
    # d = 1
    # err_fac = d*np.sqrt(np.sqrt(1+1/np.sqrt(yabs_dev)
    
    # err[mask_pos] = scl*yabs_dev[mask_pos]
    # err[mask_neg] = 1/scl
    
    # err[mask_pos] = err0*scl*yabs_dev[mask_pos]
    # err[mask_pos] = err0*scl*err_fac[mask_pos]
    err[mask_pos] = err0*scl*err_fac[mask_pos]
    err[mask_neg] = err0/scl*err_fac[mask_neg]
    # err[mask_neg] = err0/scl
    
    # err[mask_pos] = scl
    # err[mask_neg] = 1/scl/yabs_dev[mask_neg]
    # err[mask_pos] = scl/yabs_dev[mask_pos]
    # err[mask_neg] = 1/scl/yabs_dev[mask_neg]
    
    xobs_wt = xobs/err[:, np.newaxis]
    yobs_wt = yobs/err
    wt_fit = np.linalg.lstsq(xobs_wt, yobs_wt, rcond=None)
    param_wt = wt_fit[0]
    
    # N = Ndim
    # Nquad = xobs.shape[1]-N
    # # lowbnd = np.hstack((-10, np.tile(-np.inf, N), np.tile(0, N)))
    # # hibnd = np.hstack((+10, np.tile(+np.inf, N), np.tile(+np.inf, N)))
    # 
    # # lowbnd = np.hstack((np.tile(-np.inf, N), 0))
    # # lowbnd = np.hstack((np.tile(-np.inf, N), -np.inf))
    # # hibnd = np.hstack((np.tile(+np.inf, N), +np.inf))
    # lowbnd = np.hstack((np.tile(-np.inf, N), np.tile(-np.inf,Nquad )))
    # hibnd = np.hstack((np.tile(+np.inf, N), np.tile(0,Nquad)))
    # # lowbnd = np.hstack((np.tile(-np.inf, N), np.tile(-np.inf, N)))
    # 
    # wt_fit = opt.lsq_linear(xobs_wt, yobs_wt, bounds=(lowbnd, hibnd)) 
    # 
    # bnds = []
    # for ihi, ilo in zip(hibnd, lowbnd):
    #     bnds.append((ilo, ihi))
    # 
    # 
    # fun = lambda params, x=xobs_wt, y=yobs_wt: np.sum((y-np.dot(x, params))**2)
    # if param0 is None:
    #     param0 = -np.ones(Ndim+1)
    #     
    # # print(x0)
    # # print(fun(x0))
    # wt_fit = opt.minimize(fun, param0, bounds=bnds) 
    # 
    # 
    # param_wt = wt_fit['x']
    yresid_wt = yobs -  np.dot(xobs, param_wt)
    
    return param_wt, yresid_wt, err
    
def plot_resid(ind, yresid, comp, err=None, xlim=(-0.1,1.1)):
    plt.figure()
    icomp = comp.T[ind]
    
    # xmax = np.max((np.abs(np.min(icomp)), np.abs(np.max(icomp))))
    # x = np.linspace(-1.1*xmax, +1.1*xmax, 101)
    x = np.linspace(xlim[0], xlim[1], 101)
    if err is None:
        plt.plot(icomp, yresid, 'ko')
    else:
        plt.errorbar(icomp, yresid, yerr=err, fmt='ko')
        
    plt.plot(x,0*x,'r--')
    plt.xlim(xlim)
    

In [ ]:
def energy_diff(wt, yobs, comp, param_wt, cross_term_inds=cross_term_inds):
    comp_wt = np.dot(comp.T, wt)
    
    N = comp.shape[1]
    mu_pseudo_lin = np.dot(param_wt[:N], comp_wt)
    
    
    # XiXj = comp_wt[cross_term_inds[0]]*comp_wt[cross_term_inds[1]]
    # NX = XiXj.size
    # X2_sum = np.sum(XiXj)
    # mu_pseudo_curv = param_wt[-1]*X2_sum
    # mu_pseudo_curv = np.dot(param_wt[-NX:],XiXj)
    
    XlogX = comp_wt*np.log(comp_wt)
    XlogX[comp_wt==0] = 0
    XlogX_sum = np.sum(XlogX)
    mu_pseudo_curv = param_wt[-1]*XlogX_sum
    
    
    mu_endmem = np.dot(wt, yobs)
    
    dmu = mu_pseudo_lin + mu_pseudo_curv - mu_endmem
    return dmu

def endmem_subset(mask, yobs, comp):
    comp_sub = comp[mask,:]
    yobs_sub = yobs[mask]
    
    return yobs_sub, comp_sub
    
    
# def energy_jac(wt, dmu_lin, quad_terms):
#     # dmu_lin = np.dot(param_wt[:NX],comp_eig.T)
#     dmu_quad = np.dot(wt, quad_terms)
#     # dmu_endmem = yobs
#     
#     dmu_dw =  dmu_lin+dmu_quad
#     return dmu_dw

In [ ]:
yobs.shape

In [ ]:

# %%timeit
param, yresid, err = reweight_fit(1.0, xobs, yobs, yresid=0, Ndim=Ndim)
# print(param)
param_wt = param
yresid_wt = yresid

NX = comp_eig.shape[1]
print(param[:NX])
print(param[-NX:])
# param_wt[-NX:]=.01


# plot_resid(3, yresid, comp_eig, xlim=[-.5,.5])
# plot_eig_resid(3, yresid, comp_ext)
# print(yresid)

In [ ]:

expfac = (1+np.arange(20))/10
# plot_eig_resid(3, yresid, comp_eig)
param_wt = param.copy()
yresid_wt = yresid.copy()
for fac in expfac:
    # param_wt, yresid_wt, err = reweight_fit(10**fac, xobs, yobs, yresid=yresid_wt, Ndim=Ndim,
    #                                        param0=param_wt)
    # plot_eig_resid(3, yresid_wt, comp_eig)
    # param_wt, yresid_wt, err = reweight_fit(10**fac, xobs, yobs, yresid=yresid_wt, Ndim=Ndim)
    param_wt, yresid_wt, err = reweight_fit(10**fac, xobs, yobs, yresid=yresid_wt, 
                                            param0=param_wt,Ndim=Ndim)
    # param_wt, yresid_wt, err = reweight_fit(10**fac, xobs, yobs, yresid_wt, Ndim=Ndim)
    
# param_wt, yresid_wt, err = reweight_fit(10**expfac[-1], xobs, yobs, yresid_wt, Ndim=Ndim)
    
NX = comp_eig.shape[1]
print(param_wt[:NX])
print(param_wt[-1])

comp_eig= np.dot(xobs[:,:-1],vh.T)
# plot_resid(3, yresid_wt, comp_eig, xlim=[-.5,.5])
# plot_eig_resid(3, yresid_wt, comp_ext)

# print(param_wt[0])
# print(param_wt[1:NX+1])
# print(param_wt[-NX:])
# print('done')

In [ ]:
np.max(np.abs(yresid_wt))

In [ ]:
# plot_resid(3, yresid, comp_eig,xlim=[-.5,.5])

plot_resid(0, yresid_wt, comp_eig, xlim=(-.5,0.5))

In [ ]:
plot_resid(0, yresid_wt, xobs[:,:-1], xlim=(-.1,1.1))

In [ ]:
np.sum(np.sort(yresid_wt[:Npts])<.1)

In [ ]:
Npts

In [ ]:
plot_resid(3, yresid, comp_eig,xlim=[-.5,.5])

plot_resid(0, yresid_wt, comp_eig, xlim=(-.5,0.5))

In [ ]:
plot_resid(2, yresid_wt, xobs[:,:-1], xlim=(-.1,1.1))

In [ ]:
np.sum(np.sort(yresid_wt[:Npts])<.1)

In [ ]:
Npts

In [ ]:
# xobs = np.hstack((comp_eig, comp_eig**2))

In [ ]:
comp_eig.shape

In [ ]:
inds = np.argsort(yresid_wt)
# mask = yresid_wt<0
# neg_std = np.sqrt(np.mean(yresid_wt[mask]**2))
# mask = yresid_wt< 3*neg_std

neg_max = np.abs(np.min(yresid_wt))
mask = yresid_wt< +3*neg_max
plt.plot(yresid_wt[~mask], 'ko')
plt.plot(yresid_wt[mask], 'rx')
# plt.ylim(-.001,.001)

In [ ]:
np.sum(mask)

In [ ]:
yobs = yobs[inds[:40]]
xobs = xobs[inds[:40],:]

In [ ]:
# yresid_wt[mask]

In [ ]:
yobs[mask]

In [ ]:
yresid_wt[mask]

In [ ]:
param_wt.shape

In [ ]:
comp_dev.shape
mask.shape

In [ ]:
param_wt

In [ ]:
# def calc_comp_terms(comp_eig, param_wt, yobs):
#     NX = comp_eig.shape[1]
#     comp2_eig = np.zeros((comp_eig.shape[0],comp_eig.shape[0],comp_eig.shape[1]))
#     for ind in range(comp_eig.shape[1]):
#         icomp_eig = comp_eig[:,ind]
#         icomp2 = np.dot(icomp_eig[:,np.newaxis], icomp_eig[np.newaxis,:])
#         comp2_eig[:,:,ind] = icomp2
#     
#     dmu_lin = param_wt[0]+np.dot(param_wt[1:NX+1],comp_eig.T)-yobs
#     quad_terms = 2*np.dot(comp2_eig, param_wt[-NX:])
#     return comp2_eig, dmu_lin, quad_terms

In [ ]:
# comp2_eig, dmu_lin, quad_terms = calc_comp_terms(comp_eig, param_wt, yobs)

In [ ]:
yobs.size

In [ ]:
N = np.sum(mask)
N = len(yobs)
wt = np.random.rand(N)    
wt = wt/np.sum(wt)
# wt.sum()
# wt = np.zeros(N)
# wt[4] = 1

energy_diff(wt,yobs,xobs[:,:-1],param_wt)

In [ ]:
elem_comps_all.shape

In [ ]:
# dmudw = energy_jac(wt, dmu_lin, quad_terms)
# 
# dw = 1e-3
# dmudw_num = np.zeros(wt.shape)
# for ind in range(wt.size):
#     wt_num = wt.copy()
#     wt_num[ind] += dw
#     dmudw_num[ind] = (energy_diff(wt_num,yobs,comp_eig)-energy_diff(wt,yobs,comp_eig))/dw
# 

In [ ]:
# dmudw_num

In [ ]:
# plt.figure()
# plt.plot(100*(dmudw_num/dmudw-1), 'ko')
# plt.y

In [ ]:
%%timeit
energy_diff(wt, yobs, elem_comps_all)

In [ ]:
# %%timeit
# energy_jac(wt, dmu_lin, quad_terms)

In [ ]:

yobs.shape

In [ ]:
mu_uniq.size

In [ ]:
mask.sum()

In [ ]:
import scipy as sp
# sp.optimize.minimize?

In [ ]:

N = len(yobs)
yobs_sub, comp_sub = yobs, xobs[:,:-1]
# N = np.sum(mask)
# yobs_sub, comp_sub = endmem_subset(mask, yobs, elem_comps_all)

wt0 = np.random.rand(N)
wt0 = wt0/np.sum(wt0)

# comp2_eig_sub, dmu_lin_sub, quad_terms_sub = calc_comp_terms(
#     comp_eig_sub, param_wt, yobs_sub)

fun = (lambda wt, yobs=yobs_sub, comp=comp_sub, param_wt=param_wt, inds=cross_term_inds:
       energy_diff(wt, yobs, comp, param_wt, cross_term_inds=inds))

# fun = (lambda wt, yobs=yobs, comp=elem_comps_all, param_wt=param_wt, inds=cross_term_inds:
#        energy_diff(wt, yobs, comp, param_wt=param_wt, cross_term_inds=inds))

# energy_diff(wt, yobs, comp, param_wt=param_wt, cross_term_inds=cross_term_inds)

# jac = (lambda wt, dmu_lin=dmu_lin_sub, quad_terms=quad_terms_sub: 
#        energy_jac(wt, dmu_lin, quad_terms))

# def energy_diff(wt, yobs, comp_eig, param_wt=param_wt):

bnds = np.vstack((np.zeros(N),np.ones(N))).T
# fun(wt0)

A = np.ones(N)
constr = sp.optimize.LinearConstraint(A, 1.0, 1.0)

# wt0.shape
# bnds


In [ ]:
wt0.shape

In [ ]:
wt0 = np.random.rand(N)
wt0 = wt0/np.sum(wt0)

output = sp.optimize.minimize(fun, wt0, bounds=bnds, constraints=constr)
output

In [ ]:
%%timeit
    
output = sp.optimize.minimize(fun, wt0, bounds=bnds, constraints=constr)


In [ ]:
# %%timeit
#     
# output = sp.optimize.minimize(fun, wt0, bounds=bnds, constraints=constr, jac=jac)


In [ ]:

wt_fit = output['x']

plt.figure()
plt.plot(wt_fit, 'ko')
plt.title(output['fun'])

In [ ]:
# plt.plot(yresid_wt[~mask], 'ko')
yresid_wt[mask][wt_fit>.01]

In [ ]:

wt_fit = output['x']

plt.figure()
plt.plot(wt_fit, 'ko')
plt.title(output['fun'])

In [ ]:
.01533*100

In [ ]:

N = np.sum(mask)
wt0 = np.random.rand(N)
wt0 = wt0/np.sum(wt0)
# %%timeit
# output_min = sp.optimize.minimize(energy_diff, wt0, bounds=bnds, constraints=constr)

# wt0 = np.random.rand(N)
output = sp.optimize.minimize(fun, wt0, bounds=bnds, constraints=constr)
output['fun']

In [ ]:

N = len(yobs)
N

In [ ]:
# %%timeit

# output_min = sp.optimize.minimize(energy_diff, wt0, bounds=bnds, constraints=constr)
# minimizer_kwargs = 
# T = 10.0

wt0 = np.random.rand(N)
T = 3
min_kwargs = {}
min_kwargs['bounds'] = bnds
min_kwargs['constraints'] = constr

# min_kwargs['jac'] = jac

# output_bh = sp.optimize.basinhopping(
#     energy_diff, wt0, minimizer_kwargs=min_kwargs, T=T, stepsize=.2, niter=10)
output_bh = sp.optimize.basinhopping(
    fun, wt0, minimizer_kwargs=min_kwargs, T=T, stepsize=.5, niter=30)

wt_fit = output_bh['x']
funval = output_bh['fun']

shift = -output_bh['fun']
print(shift)

In [ ]:
plt.figure()
plt.plot(wt_fit, 'ko')
plt.title(output_bh['fun'])
plt.ylim(-.2, .1)
# yresid_wt[mask][wt_fit>.1]

In [ ]:
fun2 = lambda wt: fun(wt)+shift

In [ ]:
fun2(wt_fit)

In [ ]:
# %%timeit

# output_min = sp.optimize.minimize(energy_diff, wt0, bounds=bnds, constraints=constr)
# minimizer_kwargs = 
# T = 10.0

wt0 = np.random.rand(N)
T = 3
min_kwargs = {}
min_kwargs['bounds'] = bnds
min_kwargs['constraints'] = constr
min_kwargs['jac'] = jac
# output_bh = sp.optimize.basinhopping(
#     energy_diff, wt0, minimizer_kwargs=min_kwargs, T=T, stepsize=.2, niter=10)
output_bh = sp.optimize.basinhopping(
    fun2, wt0, minimizer_kwargs=min_kwargs, T=T, stepsize=.5, niter=30)

wt_fit = output_bh['x']
funval = output_bh['fun']
print(funval)


In [ ]:
plt.figure()
plt.plot(wt_fit, 'ko')
plt.title(output_bh['fun'])
yresid_wt[mask][wt_fit>.1]

In [ ]:
# T=1.0, stepsize=0.5, minimizer_kwargs=None, take_step=None, 
# accept_test=None, callback=None, interval=50, 
# disp=False, niter_success=None, seed=None

In [ ]:
output_bh

In [ ]:
wt = np.ones(yresid_wt.shape)
wt[mask] = 3

# wt = np.zeros(yresid_wt.shape)
# wt[mask] = 1

wt = np.random.rand(yresid_wt.size)
wt = wt/np.sum(wt)
comp_dev_wt = np.dot(wt, comp_dev)

comp_eig_wt = np.dot(comp_dev_wt, vh.T)
elems = avg_comp+comp_dev_wt


plt.figure()
plt.plot(wt, yobs, 'ko')

print(energy_diff(wt))
print(np.polyfit(wt, yobs,1))

# elems

In [ ]:
# np.dot(comp_dev_wt, vh.T)

In [ ]:
energy_diff(wt)

In [ ]:
wt

In [ ]:
comp_eig_wt = np.dot(comp_dev_wt, vh.T)
comp_eig_wt

In [ ]:
ind=2
plot_eig_resid(ind, yresid_wt, comp_eig)
plt.plot(comp_eig_wt[ind],0, 'ro')

In [ ]:
elems = avg_comp+np.dot(comp_dev.T,wt)
elems

In [ ]:
avg_comp +np.dot(comp_eig.T, wt)

In [ ]:

param_wt, yresid_wt, err =  reweight_fit(scl, xobs, yobs, yresid)

In [ ]:
yresid_scl = yresid/np.std(yresid)
scl = 1.0
err = np.ones(yresid_scl.shape)
err[yresid_scl>0] = scl
err[yresid_scl<0] = 1/scl

In [ ]:

scl_vals = np.logspace(0,1,5)


In [ ]:
plt.figure()
icomp_eig = comp_eig.T[2]
# plt.plot(icomp_eig, yresid_scl, 'ko')
plt.errorbar(icomp_eig, yresid_scl, yerr=err, fmt='ko')
# plt.plot(icomp_eig, yresid_scl, 'ko')
plt.plot(x,0*x,'r--')
plt.xlim(-1,1)

In [ ]:
# Aw = A * np.sqrt(W[:,np.newaxis])
# Bw = B * np.sqrt(W)
# X = np.linalg.lstsq(Aw, Bw)

X

xobs_wt = xobs/err[:, np.newaxis]
yobs_wt = yobs/err
wt_fit = np.linalg.lstsq(xobs_wt, yobs_wt)
yresid_wt = yobs -  np.dot(xobs, wt_fit[0])


plt.figure()
icomp_eig = comp_eig.T[2]
# plt.plot(icomp_eig, yresid_scl, 'ko')
# plt.errorbar(icomp_eig, yresid_wt, yerr=err, fmt='ko')
plt.plot(icomp_eig, yresid_wt, 'ko')
plt.plot(x,0*x,'r--')
plt.xlim(-1,1)

In [ ]:
obs = np.hstack((mu_dev_uniq[:,np.newaxis]/1e5, comp_eig, comp_eig**2))

reg = LinearRegression()
reg.fit(obs[:,1:], obs[:,0])

reg.coef_


resid = obs[:,0] - reg.predict(obs[:,1:])

np.std(resid*1e5)

In [ ]:
plt.figure()
icomp_eig = comp_eig.T[2]
plt.plot(icomp_eig, resid*1e5/1e3, 'ko')

In [ ]:

obs_resid = np.hstack((resid[:,np.newaxis], comp_eig, comp_eig**2))
obs_resid.shape

In [ ]:
hibnd = -np.ones(obs_resid.shape[0])
lobnd = +np.ones(obs_resid.shape[0])
y = np.hstack((hibnd, lobnd))

fac = 10.0
X = np.vstack((obs_resid/fac, obs_resid*fac))

In [ ]:
y.shape
reg = LogisticRegression(fit_intercept=False, solver='liblinear', warm_start=True)

In [ ]:
# prop_terms
# prop_terms.shape
reg.fit(X, y)
params = np.squeeze(reg.coef_)
# params[0]
params_scl = params/params[0]
params_scl

In [ ]:
mumod = np.dot(obs_resid[:,1:], params_scl[1:])


In [ ]:
plt.figure()
icomp_eig = comp_eig.T[9]
plt.plot(icomp_eig, -mumod*1e5/1e3, 'ko')
xl = plt.xlim()
plt.plot(xl,[0,0], 'r-')

In [ ]:
# elem_comps_uniq.shape

In [ ]:
# icomp[np.newaxis,:] * icomp[:, np.newaxis]
lin_terms = elem_comps_uniq
lin_terms.shape

In [ ]:
quad_terms = elem_comps_uniq[:,np.newaxis,:]*elem_comps_uniq[:,:,np.newaxis]
shp = quad_terms.shape


In [ ]:
shp

In [ ]:
# offset_terms = np.ones(shp[0])[:,np.newaxis]
# offset_terms.shape

In [ ]:
quad_terms_flat = np.reshape(quad_terms, (shp[0],shp[1]*shp[2]))
# quad_terms_flat[17] -quad_terms[17].ravel()
quad_terms_flat.shape



In [ ]:
all_terms = np.hstack((lin_terms, quad_terms_flat))
all_terms.shape

In [ ]:
ind_rows, ind_cols = np.tril_indices(shp[0],-1)
pt_pairs = np.vstack((ind_rows,ind_cols)).T
# pt_pairs

In [ ]:
pt_pairs.shape

In [ ]:
midpt_terms = np.mean(all_terms[pt_pairs],axis=1)
midpt_mu = np.mean(mu_uniq[pt_pairs],axis=1)

In [ ]:

from sklearn.linear_model import LinearRegression, LogisticRegression

from sklearn import linear_model

In [ ]:
avg_mu = np.mean(midpt_mu)
np.std(midpt_mu-avg_mu)

In [ ]:
midpt_terms.shape

In [ ]:
reg = LinearRegression()

ideal_terms = expand_terms[:, :len(sys_elems)]

In [ ]:

reg.fit(ideal_terms, expand_mu)
reg.coef_
ideal_mu = reg.predict(ideal_terms)
dmu = expand_mu-ideal_mu
np.std(dmu)/1e3

In [ ]:
hibnd = -np.ones(expand_terms.shape[0])
lobnd = +np.ones(expand_terms.shape[0])

prop_terms = np.hstack((dmu[:,np.newaxis]/1e6, expand_terms))
fac =10
X = np.vstack((prop_terms*fac, prop_terms/fac))
y = np.hstack((hibnd, lobnd))

In [ ]:
Nparams = prop_terms.shape[1]
Nparams
coef0 = np.zeros(Nparams)
coef0[0] = 1

In [ ]:
# prop_terms
# prop_terms.shape
reg = LogisticRegression(fit_intercept=False, solver='liblinear')
reg.fit(X, y)
reg.coef_
reg.coef_

In [ ]:
# np.linalg.lstsq?

In [ ]:
y = 2*np.random.randint(2,size=expand_mu.size)-1

In [ ]:
# reg = LinearRegression().fit(all_terms, mu_uniq)

reg = LogisticRegression(fit_intercept=True)

# 
# reg.score(all_terms, mu_uniq)

In [ ]:

reg.fit(expand_terms, y)

reg.coef_

In [ ]:
reg.fit_intercept=False
reg.intercept_ = np.random.randn(expand_mu.size)

reg.fit(expand_terms, y)

In [ ]:
reg.intercept_

In [ ]:
reg = Regression().fit(expand_terms, expand_mu)
reg.coef_

In [ ]:


# reg.coef_
# 
# reg.intercept_ 

# mu_uniq-reg.predict(all_terms)
(expand_mu-reg.predict(expand_terms))/1e3


In [ ]:
reg = linear_model.Ridge(alpha=1e0, max_iter=1e4)
reg.fit(all_terms, mu_uniq)


# reg.coef_
reg.score(all_terms, mu_uniq)
mu_uniq-reg.predict(all_terms)

In [ ]:
reg = linear_model.Lasso(alpha=0.5,max_iter=1e4)
reg.fit(all_terms, mu_uniq)


reg.coef_

In [ ]:
np.linalg.lstsq(all_terms, mu_uniq)

Now contruct the convex hull, using the extra bulk composition point as a viewpoint below the hull.  
The 'OJ' option is required to avoid roundoff errors that inhibit contruction; remove the option and run the code for a complete explanation.  
The 'OGn' option locates the viewpoint, which does not otherwise contribute to hull construction. 

In [ ]:
hull = sci.spatial.ConvexHull(points, qhull_options='QJ QG'+str(points.shape[0]-1))

In [ ]:
hull.simplices.shape, hull.vertices.shape, hull.points.shape

In [ ]:
hull.simplices

In [ ]:
def get_pseudo_phase(T, P, phases, offset=0, oxide_basis=False,
                     test_endmember=False, test_solution=False):
    
    
    
    

## Phases in Stixrude
Instantiate the database and optionally, print an info table

#### Create a Phase object for each phase in the database
Lode this information into dictionaries and lists for future reference

# Construct pseudophase

Aaron's notes:

1. Calculate the lower convex hull for the pure and endmember phases.
2. Adopt a modified ideal solution, where the mixing contribution is given by a scaled ideal entropy $-c RT \sum_i X_i \log X_i$.
3. Endmember chemical potentials as well as the scale factor $c$, are determined by least-squares fitting.
4. Endmember chemical potentials must be adjusted upwards to guarantee that the phase is everywhere metastable.
4.1 Endmember potentials are shifted so that every vertex of the convex hull lies on or below the omnicomponent surface, ensuring that all the equilibrium pure and endmember phases are individually stable relative to the omnicomponent phase.
4.2 Calculate center point of the hull vertices, add $X_i \sum_i \log X_i$ to the hull at thgis point, and insure that omnicomponent phase has an energy equal to or above this energy
5. Finally add an additional 1 J to each endmember potential just to insure numerical stability.

### (1) composition matrix
Relate endmember of each phase (rows) to moles of elements (columns):
- columns are indexed on atomic number
- rows are indexed on phase order, listed above

In [ ]:
C = []
row_names = []
for phase in pure_phases:
    if use_oxides_as_basis:
        C.append(core.chem.calc_mol_oxide_comp(phase.props['element_comp'][0]))
    else:
        C.append(phase.props['element_comp'][0])
    row_names.append(phase.props['endmember_name'][0])
for phase in soln_phases:
    for i in range(0,phase.endmember_num):
        if use_oxides_as_basis:
            C.append(core.chem.calc_mol_oxide_comp(phase.props['element_comp'][i]))
        else:
            C.append(phase.props['element_comp'][i])
        row_names.append(phase.props['endmember_name'][i])
C = np.array(C)
C.shape

Filter for the non-zero abundance elements/oxides in the system

In [ ]:
elm_sys_ind = np.where(np.sum(C,axis=0) > 0)[0]
if use_oxides_as_basis:
    elm_sys = [core.chem.oxide_props['oxides'][i] for i in elm_sys_ind]
else:
    elm_sys = [core.chem.PERIODIC_ORDER[i] for i in elm_sys_ind]
elm_sys

Deflate the composition matrix:
- columns correspond to non-zero elemental abundances in teh system
- rows are as previous

In [ ]:
C = C[:,elm_sys_ind]
C.shape

### (2) make a vector of chemical potentials of each endmember

In [ ]:
mu = []
for pureph in pure_phases:
    mu += [pureph.gibbs_energy(t,p)]
for solnph in soln_phases:
    mu += [solnph.gibbs_energy(t,p,mol=np.eye(solnph.endmember_num)[i],deriv={"dmol":1})[0,i] for i in range(0,solnph.endmember_num)]
mu = np.array(mu)
mu.shape

### (3) Convex Hull construction
- Hull construction depends ONLY on C and mu as defined above
- We define an additional multidimensional point called a viewpoint from which we can ask the question "what hull facets are viewable from that viewpoint?" 
- If the viewpoint is the bulk composition of the system, and we give the viewpoint an energy lower than any of the computed values of mu, then we should be able to "see" from that viewpoint all the hull facets that define the lowest energy polyhedron.
- The vertices of these facets are the "active" phases from which we can construct the pseudo-omnicomponent phase

For the system bulk composition, we use the average composition of the system, normalized to one mole.  
For the chemical potential we use 1.5 times the most negative chemical potential of any endmember in the system

#### Reduce the C matrix to unique rows (unique phase stoichiometry)
The c_inverse_array maps the indices of C_unique back to the full C matrix

In [ ]:
C_unique,C_inverse_ind = np.unique(C, axis=0,return_inverse=True)
C_unique.shape,C_inverse_ind.shape

Examine the chemical potentials of all stoichiometrically redundant phases and load an array with the most negative chemical potential (the stablest) of all the values found.

In [ ]:
mu_unique = []
for i in range(0,C_unique.shape[0]):
     mu_unique.append(np.min(mu[np.where(C_inverse_ind == i)]))
mu_unique = np.array(mu_unique)

Next, scale all the rows for one mole of each phase

In [ ]:
for i in range(0,C_unique.shape[0]):
    sum = np.sum(C_unique[i,:])
    C_unique[i,:] /= sum
    mu_unique[i] /= sum

Compute an average composition and assign it a chemical potentials 1.5 times more negative than the most negative unique endmember

In [ ]:
blk_cmp = np.sum(C_unique,axis=0)
blk_cmp = blk_cmp/np.sum(blk_cmp)
print ('Average composition of the viewpoint:', blk_cmp)
mu_blk_cmp = np.min(mu_unique)*1.5
print ('Chemical potential of the viewpoint:', mu_blk_cmp)

Now contruct the convex hull, using the extra bulk composition point as a viewpoint below the hull.  
The 'OJ' option is required to avoid roundoff errors that inhibit contruction; remove the option and run the code for a complete explanation.  
The 'OGn' option locates the viewpoint, which does not otherwise contribute to hull construction. 

In [ ]:
point_A = np.vstack((C_unique,blk_cmp))
point_B = np.vstack((np.reshape(mu_unique,(mu_unique.shape[0],1)),np.array([mu_blk_cmp])))
points = np.hstack((point_A,point_B))
hull = sci.spatial.ConvexHull(points, qhull_options='QJ QG'+str(points.shape[0]-1))

In [ ]:
hull.simplices.shape, hull.vertices.shape, hull.points.shape

### (4) Cull the hull
Determine which rows (phase endmembers) contribute to the lower most hull (as seen by the bulk composition)

In [ ]:
act_ind = np.full(mu_unique.shape, False, dtype=bool)
for visible_facet in hull.simplices[hull.good]:
    for index in visible_facet:
        act_ind[index] = True
for i,v in enumerate(act_ind):
    if not v:
        print(row_names[i], 'is not on the lower hull')
act_ind

Remove rows from C and mu, that is remove the phases that cannot be "seen" from the viewpoint

In [ ]:
mu = mu_unique[act_ind]
mu.shape

In [ ]:
C = C_unique[act_ind,:]
C.shape

### (5) solve for the internally consistent chemical potenials of the elements
Note that the endmember chemical potentials are adjusted by the pseudo-phase solution entropy.  This insures that the regression yields endmember potentials for the chemical elements that are consistent with ideal mixing.

In [ ]:
Cplus = np.hstack((C,np.zeros((C.shape[0],1))))
for i in range(0,C.shape[0]):
    sum = np.sum(C[i,:])
    s = 0.0
    for j in range(0,C.shape[1]):
        if C[i,j] > 0:
            X = C[i,j]/sum
            s += X*np.log(X)
    Cplus[i,-1] = s*sum*8.3143*t

The last regression parameter, $m$, is a multiplicity factor on the ideal entropy, while the other terms, $\mu _i^{o,elm}$, are the endmember chemical potentials of the pseudo-phase. The endmembers are the chemical elements.  I.e.,  
${\hat G^{pseudo}} = \sum\limits_i^{elm} {X_i^{elm}} \mu _i^{o,elm} + mRT\sum\limits_i^{elm} {X_i^{elm}\ln } X_i^{elm}$

In [ ]:
x,residuals,rank,s = np.linalg.lstsq(Cplus,mu,rcond=None)
x,np.sqrt(residuals),rank,s

Apply the offset to the endmember chemical potentials

In [ ]:
for i in range(0,x.shape[0]-1):
    x[i] += offset_value

## Build endmembers of pseudo-phase using the coder module

In [ ]:
modelCD = coder.StdStateModel()

In [ ]:
GTP = sym.symbols('GTP')
params = [('GTP','J',GTP)]
modelCD.add_expression_to_model(GTP, params)

In [ ]:
modelCD.set_module_name('pseudo_end')

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

In [ ]:
def standardize_formula(form):
    cmp = form.split('O')
    str = ''
    if cmp[0][-1].isdigit():
        str += cmp[0][:-1] + '(' + cmp[0][-1] + ')'
    else:
        str += cmp[0] + '(1)'
    if cmp[1] == '':
        str += 'O'
    else:
        str += 'O(' + cmp[1] + ')'
    return str

In [ ]:
model_type = "calib"
for ind,elm in enumerate(elm_sys):
    if use_oxides_as_basis:
        formula = standardize_formula(elm)
    else:
        formula = elm+'(1)'
    param_dict = {'Phase':elm,'Formula':formula,'T_r':298.15,'P_r':1.0,'GTP':x[ind]}
    print (param_dict)
    result = modelCD.create_code_module(phase=param_dict.pop('Phase', None),
                                      formula=param_dict.pop('Formula', None),
                                      params=param_dict,
                                      module_type=model_type,
                                      silent=True)
    print ('Component', elm, 'done!')

Build the code (ignore error messages generated by Cython regarding 'language_level')

In [ ]:
import pseudo_end
%cd ..

## Test the endmember code

In [ ]:
if test_endmember_code:
    print ('Endmember metadata:')
    try:
        print(pseudo_end.cy_Fe_pseudo_end_calib_identifier())
        print(pseudo_end.cy_Fe_pseudo_end_calib_name())
        print(pseudo_end.cy_Fe_pseudo_end_calib_formula())
        print(pseudo_end.cy_Fe_pseudo_end_calib_mw())
        print(pseudo_end.cy_Fe_pseudo_end_calib_elements())
    except AttributeError:
        pass
    fmt = "{0:<10.10s} {1:13.6e} {2:<10.10s}"
    print ('Thermodynamic properties:')
    try:
        print(fmt.format('G', pseudo_end.cy_Fe_pseudo_end_calib_g(t,p), 'J/m'))
        print(fmt.format('dGdT', pseudo_end.cy_Fe_pseudo_end_calib_dgdt(t,p), 'J/K-m'))
        print(fmt.format('dGdP', pseudo_end.cy_Fe_pseudo_end_calib_dgdp(t,p), 'J/bar-m'))
        print(fmt.format('d2GdP2', pseudo_end.cy_Fe_pseudo_end_calib_d2gdt2(t,p), 'J/K^2-m'))
        print(fmt.format('d2GdTdP', pseudo_end.cy_Fe_pseudo_end_calib_d2gdtdp(t,p), 'J/K-bar-m'))
        print(fmt.format('d2GdP2', pseudo_end.cy_Fe_pseudo_end_calib_d2gdp2(t,p), 'J/bar^2-m'))
        print(fmt.format('d3GdT3', pseudo_end.cy_Fe_pseudo_end_calib_d3gdt3(t,p), 'J/K^3-m'))
        print(fmt.format('d3GdT2dP', pseudo_end.cy_Fe_pseudo_end_calib_d3gdt2dp(t,p), 'J/K^2-bar-m'))
        print(fmt.format('d3GdTdP2', pseudo_end.cy_Fe_pseudo_end_calib_d3gdtdp2(t,p), 'J/K-bar^2-m'))
        print(fmt.format('d3GdP3', pseudo_end.cy_Fe_pseudo_end_calib_d3gdp3(t,p), 'J/bar^3-m'))
        print(fmt.format('S', pseudo_end.cy_Fe_pseudo_end_calib_s(t,p), 'J/K-m'))
        print(fmt.format('V', pseudo_end.cy_Fe_pseudo_end_calib_v(t,p), 'J/bar-m'))
        print(fmt.format('Cv', pseudo_end.cy_Fe_pseudo_end_calib_cv(t,p), 'J/K-m'))
        print(fmt.format('Cp', pseudo_end.cy_Fe_pseudo_end_calib_cp(t,p), 'J/K-m'))
        print(fmt.format('dCpdT', pseudo_end.cy_Fe_pseudo_end_calib_dcpdt(t,p), 'J/K^2-m'))
        print(fmt.format('alpha', pseudo_end.cy_Fe_pseudo_end_calib_alpha(t,p), '1/K'))
        print(fmt.format('beta', pseudo_end.cy_Fe_pseudo_end_calib_beta(t,p), '1/bar'))
        print(fmt.format('K', pseudo_end.cy_Fe_pseudo_end_calib_K(t,p), 'bar'))
        print(fmt.format('Kp', pseudo_end.cy_Fe_pseudo_end_calib_Kp(t,p), ''))
    except AttributeError:
        pass
    print ('Parameters:')
    try:
        npar = pseudo_end.cy_Fe_pseudo_end_get_param_number()
        names = pseudo_end.cy_Fe_pseudo_end_get_param_names()
        units = pseudo_end.cy_Fe_pseudo_end_get_param_units()
        values = pseudo_end.cy_Fe_pseudo_end_get_param_values()
        fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
        for i in range(0,npar):
            print(fmt.format(names[i], values[i], pseudo_end.cy_Fe_pseudo_end_get_param_value(i), units[i]))
    except AttributeError:
        pass
    try:
        values[1] = 100.0
        pseudo_end.cy_Fe_pseudo_end_set_param_values(values)
        fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
        for i in range(0,npar):
            print(fmt.format(names[i], values[i], pseudo_end.cy_Fe_pseudo_end_get_param_value(i), units[i]))
    except (AttributeError, NameError):
        pass
    try:
        pseudo_end.cy_Fe_pseudo_end_set_param_value(1, 1.0)
        fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
        for i in range(0,npar):
            print(fmt.format(names[i], values[i], pseudo_end.cy_Fe_pseudo_end_get_param_value(i), units[i]))
    except AttributeError:
        pass

## Build solution pseudo-phase using the coder module
This code utilizes the previous endmember models generated above. 

In [ ]:
c = len(elm_sys)

In [ ]:
modelCD = coder.SimpleSolnModel(nc=c)

In [ ]:
n = modelCD.n
nT = modelCD.nT
X = n/nT

In [ ]:
T = modelCD.get_symbol_for_t()
mu = modelCD.mu

In [ ]:
G_ss = (n.transpose()*mu)[0]
G_ss

In [ ]:
S_config,R,multiplier = sym.symbols('S_config R multiplier')
S_config = 0
for i in range(0,c):
    S_config += X[i]*sym.log(X[i])
S_config *= -R*nT*multiplier

In [ ]:
G_config = sym.simplify(-T*S_config)
G_config

In [ ]:
G = G_ss + G_config

In [ ]:
modelCD.add_expression_to_model(G, [('multiplier', 'none', multiplier)])

In [ ]:
modelCD.module = "pseudo_soln"

In [ ]:
formula = ''
convert = []
test = []
if use_oxides_as_basis:
    for ind,elm in enumerate(elm_sys):
        ox_index = list(core.chem.oxide_props['oxides']).index(elm)
        ox_cat = core.chem.oxide_props['cations'][ox_index]
        formula += ox_cat + '[' + ox_cat + ']'
        ox_cat_num = core.chem.oxide_props['cat_num'][ox_index]
        if ox_cat_num > 1:
            convert.append('['+str(ind)+']=['+ox_cat+']/'+str(ox_cat_num)+'.0')
        else:
            convert.append('['+str(ind)+']=['+ox_cat+']')
        test.append('['+str(ind)+'] >= 0.0')
    formula += 'O[O]'
else:
    for ind,elm in enumerate(elm_sys):
        formula += elm + '[' + elm + ']'
        convert.append('['+str(ind)+']=['+elm+']')
        test.append('['+str(ind)+'] >= 0.0')
formula, convert, test

In [ ]:
modelCD.formula_string = formula
modelCD.conversion_string = convert
modelCD.test_string = test

In [ ]:
paramValues = {'multiplier':x[-1],'T_r':298.15,'P_r':1.0,}
endmembers = []
for elm in elm_sys:
    endmembers.append(str(elm)+'_pseudo_end')

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

In [ ]:
modelCD.create_code_module(phase="PseudoPhase", params=paramValues, endmembers=endmembers, 
                         prefix="cy", module_type='calib', silent=False)

In [ ]:
import pseudo_soln
%cd ..

## Test the solution
Characteristics of the solution

In [ ]:
if test_solution_code:
    mol = blk_cmp
    print ('Solution metadata:')
    try:
        print(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_identifier())
        print(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_name())
        print(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_formula(t,p,mol))
    except AttributeError:
        pass
    print ('Elemental conversion routines')
    try:
        e = np.zeros(106)
        sum = np.sum(mol)
        for index in range(0,c):
            end = pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_elements(index)
            for i in range(0,106):
                e[i] += end[i]*mol[index]/sum
        nConv = pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_conv_elm_to_moles(e)
        for i in range(0,c):
            print ('X[{0:d}] input {1:13.6e}, calc {2:13.6e}, diff {3:13.6e}'.format(
                i, mol[i]/sum, nConv[i], nConv[i]-mol[i]/sum))
        if not pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_test_moles(nConv):
            print ('Output of intrinsic composition calculation fails tests for permissible values.')
    except AttributeError:
        pass
    print ('Composition conversion routines')
    try:
        print (pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_conv_moles_to_tot_moles(mol))
        print (pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_conv_moles_to_mole_frac(mol))
        e = pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_conv_moles_to_elm(mol)
        print (e)
        print (pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_conv_elm_to_moles(e))
        print (pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_conv_elm_to_tot_moles(e))
        print (pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_conv_elm_to_tot_grams(e))
    except AttributeError:
        pass
    print ('Simple thermodynamic functions')
    fmt = "{0:<10.10s} {1:13.6e} {2:<10.10s}"
    try:
        print(fmt.format('G', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_g(t,p,mol), 'J'))
        print(fmt.format('dGdT', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_dgdt(t,p,mol), 'J/K'))
        print(fmt.format('dGdP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_dgdp(t,p,mol), 'J/bar'))
        print(fmt.format('d2GdT2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d2gdt2(t,p,mol), 'J/K^2'))
        print(fmt.format('d2GdTdP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d2gdtdp(t,p,mol), 'J/K-bar'))
        print(fmt.format('d2GdP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d2gdp2(t,p,mol), 'J/bar^2'))
        print(fmt.format('d3GdT3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdt3(t,p,mol), 'J/K^3'))
        print(fmt.format('d3GdT2dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdt2dp(t,p,mol), 'J/K^2-bar'))
        print(fmt.format('d3GdTdP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdtdp2(t,p,mol), 'J/K-bar^2'))
        print(fmt.format('d3GdP3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdp3(t,p,mol), 'J/bar^3'))
        print(fmt.format('S', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_s(t,p,mol), 'J/K'))
        print(fmt.format('V', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_v(t,p,mol), 'J/bar'))
        print(fmt.format('Cv', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_cv(t,p,mol), 'J/K'))
        print(fmt.format('Cp', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_cp(t,p,mol), 'J/K'))
        print(fmt.format('dCpdT', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_dcpdt(t,p,mol), 'J/K^2'))
        print(fmt.format('alpha', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_alpha(t,p,mol), '1/K'))
        print(fmt.format('beta', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_beta(t,p,mol), '1/bar'))
        print(fmt.format('K', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_K(t,p,mol), 'bar'))
        print(fmt.format('Kp', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_Kp(t,p,mol), ''))
    except AttributeError:
        pass
    print ('Endmember properties')
    fmt = "{0:<10.10s} {1:13.6e} {2:<15.15s}"
    try:
        print ("number of components", pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_number())
        for index in range(0, c):
            print ("{0:<20.20s}".format(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_name(index)), end=' ')
            print ("{0:<20.20s}".format(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_formula(index)), end=' ')
            print ("mw: {0:10.2f}".format(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_mw(index)))
            print (fmt.format('mu0', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_mu0(index,t,p), 'J/mol'))
            print (fmt.format('dmu0dT', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_dmu0dT(index,t,p), 'J/K-mol'))
            print (fmt.format('dmu0dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_dmu0dP(index,t,p), 'J/bar-mol'))
            print (fmt.format('d2mu0dT2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_d2mu0dT2(index,t,p), 'J/K^2-mol'))
            print (fmt.format('d2mu0dTdP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_d2mu0dTdP(index,t,p), 'J/K-bar-mol'))
            print (fmt.format('d2mu0dP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_d2mu0dP2(index,t,p), 'J/bar^2-mol'))
            print (fmt.format('d3mu0dT3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_d3mu0dT3(index,t,p), 'J/K^3-mol'))
            print (fmt.format('d3mu0dT2dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_d3mu0dT2dP(index,t,p), 'J/K^2-bar-mol'))
            print (fmt.format('d3mu0dTdP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_d3mu0dTdP2(index,t,p), 'J/K-bar^2-mol'))
            print (fmt.format('d3mu0dP3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_d3mu0dP3(index,t,p), 'J/bar^3-mol'))
            print ("Element array:")
            print (pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_elements(index))
            print ()
    except AttributeError:
        pass
    print ('Species properties:')
    fmt = "{0:<10.10s} {1:13.6e} {2:<15.15s}"
    try:
        print ("number of species", pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_species_number())
        for index in range(0, c):
            print ("{0:<20.20s}".format(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_species_name(index)), end=' ')
            print ("{0:<20.20s}".format(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_species_formula(index)), end=' ')
            print ("mw: {0:10.2f}".format(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_species_mw(index)))
            print ("Element array:")
            print (pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_species_elements(index))
            print ()
    except AttributeError:
        pass
    print ('First compositional derivatines:')
    def printResult(name, result, units):
        print ("{0:<10.10s}".format(name), end=' ')
        [print ("{0:13.6e}".format(x), end=' ') for x in result]
        print ("{0:<10.10s}".format(units))
    def printLabels(n):
        print ("{0:<18.18s}".format(''), end=' ')
        [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(len(n))]
        print ()
    printLabels(mol)
    try:
        printResult('dGdn', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_dgdn(t,p,mol), 'J/m')
        printResult('d2GdndT', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d2gdndt(t,p,mol), 'J/K-m')
        printResult('d2GdndP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d2gdndp(t,p,mol), 'J/bar-m')
        printResult('d3GdndT2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdndt2(t,p,mol), 'J/K^2-m')
        printResult('d3GdndTdP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdndtdp(t,p,mol), 'J/K-bar-m')
        printResult('d3GdndP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdndp2(t,p,mol), 'J/bar^2-m')
        printResult('d4GdndT3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdndt3(t,p,mol), 'J/K^3-m')
        printResult('d4GdndT2dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdndt2dp(t,p,mol), 'J/K^2-bar-m')
        printResult('d4GdndTdP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdndtdp2(t,p,mol), 'J/K-bar^2-m')
        printResult('d4GdndP3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdndp3(t,p,mol), 'J/bar^3-m')
    except AttributeError:
        pass 
    print ('Second compositional derivatives:')
    def printResult(name, result, units):
        print ("{0:<10.10s}".format(name), end=' ')
        [print ("{0:13.6e}".format(x), end=' ') for x in result]
        print ("{0:<10.10s}".format(units))
    def printLabels(n):
        print ("{0:<18.18s}".format(''), end=' ')
        maxIdx = int(len(n)*(len(n)-1)/2 + len(n))
        [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(maxIdx)]
        print ()
    printLabels(mol)
    try:
        printResult('d2Gdn2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d2gdn2(t,p,mol), 'J/m^2')
        printResult('d3Gdn2dT', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdn2dt(t,p,mol), 'J/K-m^2')
        printResult('d3Gdn2dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdn2dp(t,p,mol), 'J/bar-m^2')
        printResult('d4Gdn2dT2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdn2dt2(t,p,mol), 'J/K^2-m^2')
        printResult('d4Gdn2dTdP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdn2dtdp(t,p,mol), 'J/K-bar-m^2')
        printResult('d4Gdn2dP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdn2dp2(t,p,mol), 'J/bar^2-m^2')
        printResult('d5Gdn2dT3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d5gdn2dt3(t,p,mol), 'J/K^3-m^2')
        printResult('d5Gdn2dT2dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d5gdn2dt2dp(t,p,mol), 'J/K^2-bar-m^2')
        printResult('d5Gdn2dTdP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d5gdn2dtdp2(t,p,mol), 'J/K-bar^2-m^2')
        printResult('d5Gdn2dP3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d5gdn2dp3(t,p,mol), 'J/bar^3-m^2')
    except AttributeError:
        pass
    print ('Third compositional derivatives:')
    def printResult(name, result, units):
        print ("{0:<10.10s}".format(name), end=' ')
        [print ("{0:10.3e}".format(x), end=' ') for x in result]
        print ("{0:<14.14s}".format(units))
    def printLabels(n):
        print ("{0:<15.15s}".format(''), end=' ')
        maxIdx = int(len(n)*(len(n)+1)*(len(n)+2)/6)
        [print ("[{0:3d}]{1:<5.5s}".format(idx, ''), end=' ') for idx in range(maxIdx)]
        print ()
    printLabels(mol)
    try:
        printResult('d3Gdn3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdn3(t,p,mol), 'J/m^3')
        printResult('d4Gdn3dT', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdn3dt(t,p,mol), 'J/K-m^3')
        printResult('d4Gdn3dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdn3dp(t,p,mol), 'J/bar-m^3')
        printResult('d5Gdn3dT2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d5gdn3dt2(t,p,mol), 'J/K^2-m^3')
        printResult('d5Gdn3dTdP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d5gdn3dtdp(t,p,mol), 'J/K-bar-m^3')
        printResult('d5Gdn3dP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d5gdn3dp2(t,p,mol), 'J/bar^2-m^3')
        printResult('d6Gdn3dT3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d6gdn3dt3(t,p,mol), 'J/K^3-m^3')
        printResult('d6Gdn3dT2dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d6gdn3dt2dp(t,p,mol), 'J/K^2-bar-m^3')
        printResult('d6Gdn3dTdP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d6gdn3dtdp2(t,p,mol), 'J/K-bar^2-m^3')
        printResult('d6Gdn3dP3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d6gdn3dp3(t,p,mol), 'J/bar^3-m^3')
    except AttributeError:
        pass

## Import model for the pseudo-phase into the ThermoEngine package

In [ ]:
#%cd working
#import pseudo_soln
#%cd ..
modelPseudo = model.Database(database="CoderModule", calib="calib", 
                         phase_tuple=('pseudo_soln', {'Psu':['PseudoPhase','solution']}))
Pseudo = modelPseudo.get_phase('Psu')

for phase_name, abbrv in zip(modelPseudo.phase_info.phase_name,modelPseudo.phase_info.abbrev):
    print ('Abbreviation: {0:<10s} Name: {1:<30s}'.format(abbrv, phase_name))

Check pseudo-phase import by printning some phase characteristics

In [ ]:
print (Pseudo.props['phase_name'])
print (Pseudo.props['formula'])
print (Pseudo.props['molwt'])
print (Pseudo.props['abbrev'])
print (Pseudo.props['endmember_num'])
print (Pseudo.props['endmember_name'])

## Try the equiibrium calculations with the omnicomponent pseudo-phase
#### Choose a phase assemblage

In [ ]:
#stix_phases.keys()
phs_sys  = [Pseudo]
phs_sys += [stix_phases['Fsp'], stix_phases['Ol'], stix_phases['Cpx'], stix_phases['Grt']] # solutiopns,
phs_sys += [stix_phases['Qz'], stix_phases['Ky'], stix_phases['Nph']]
#
#phs_sys  = [Pseudo, stix_phases['Opx']]

In [ ]:
equil = equilibrate.Equilibrate(['O','Na','Mg','Al','Si','Ca','Fe'], phs_sys)

#### Set the bulk composition of the system
Input is a bulk peridotite suggested by Stixrude and Lithgow-Bertelloni

In [ ]:
grm_oxides = {
    'SiO2':  45.47, 
    'Al2O3':  4.0, 
    'FeO':    7.22, 
    'MgO':   38.53, 
    'CaO':    3.59, 
    'Na2O':   0.31
}
blk_cmp = np.zeros(7)
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
blk_cmp[0] = 2.0*mol_oxides[0] + 3.0*mol_oxides[2] + mol_oxides[5] + mol_oxides[7] + mol_oxides[10] + mol_oxides[11] # oxygen
blk_cmp[1] = 2.0*mol_oxides[11] # sodium
blk_cmp[2] = mol_oxides[7]      # magnesium
blk_cmp[3] = 2.0*mol_oxides[2]  # aluminum
blk_cmp[4] = mol_oxides[0]      # silicon
blk_cmp[5] = mol_oxides[10]     # calcium
blk_cmp[6] = mol_oxides[5]      # iron

In [ ]:
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0)
state.print_state()